In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [2]:
tokinizer=AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model=AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

C:\ProgramData\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91789\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [16]:
tokens=tokinizer.encode('I love you.',return_tensors='pt')


In [17]:
tokens

tensor([[  101,   151, 11157, 10855,   119,   102]])

In [18]:
tokinizer.decode(tokens[0])

'[CLS] i love you. [SEP]'

In [19]:
result=model(tokens)

In [20]:
result.logits

tensor([[-2.2140, -2.2489, -0.8243,  1.1694,  3.2718]],
       grad_fn=<AddmmBackward0>)

In [21]:
int(torch.argmax(result.logits))+1

5

In [23]:
r=requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup=BeautifulSoup(r.text,'html.parser')
regex=re.compile('.*comment.*')
results=soup.find_all('p',{'class':regex})
reviews=[result.text for result in results]

In [28]:
reviews[9]

'Great staff and food.  Must try is the pan fried Gnocchi!  The staff were really friendly and the coffee was good as well'

In [29]:
import pandas as pd

In [32]:
df=pd.DataFrame(np.array(reviews),columns=['review'])

In [35]:
df.tail()

,review
5,Great food amazing coffee and tea. Short walk ...
6,It was ok. Had coffee with my friends. I'm new...
7,Ricotta hot cakes! These were so yummy. I ate ...
8,We came for brunch twice in our week-long visi...
9,Great staff and food. Must try is the pan fri...


In [37]:
df['review'].iloc[2]

"Great service, lovely location, and really amazing food. Words don't do justice. We had the mushroom parm bruschetta and the loaded double double. Both were so tasty. Also love the Aussie black tea and a flat white. Wish I had more mornings in Sydney to eat breakfast there. Highly recommend."

In [38]:
def sentiment_score(review):
    tokens=tokinizer.encode(review,return_tensors='pt')
    result=model(tokens)
    return int(torch.argmax(result.logits))+1

In [41]:
sentiment_score(df['review'].iloc[1])

4

In [42]:
df["sentiment"]=df['review'].apply(lambda x: sentiment_score(x[:512]))

In [44]:
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,Six of us met here for breakfast before our wa...,4
2,"Great service, lovely location, and really ama...",5
3,Great place with delicious food and friendly s...,5
4,Some of the best Milkshakes me and my daughter...,5
5,Great food amazing coffee and tea. Short walk ...,5
6,It was ok. Had coffee with my friends. I'm new...,3
7,Ricotta hot cakes! These were so yummy. I ate ...,5
8,We came for brunch twice in our week-long visi...,4
9,Great staff and food. Must try is the pan fri...,5
